In [2]:
import requests
import json
import pandas as pd
from config import api_key

In [28]:
url = "https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=GOLD&interval=5min&apikey=demo&outputsize=full"
api_key = "&apikey=" + api_key

In [29]:
response = requests.get(url + api_key).json()
print(json.dumps(response, indent=4, sort_keys=True))

{
    "Meta Data": {
        "1. Information": "Intraday (5min) open, high, low, close prices and volume",
        "2. Symbol": "GOLD",
        "3. Last Refreshed": "2023-02-03 20:00:00",
        "4. Interval": "5min",
        "5. Output Size": "Full size",
        "6. Time Zone": "US/Eastern"
    },
    "Time Series (5min)": {
        "2023-01-09 04:10:00": {
            "1. open": "19.1700",
            "2. high": "19.1700",
            "3. low": "19.1600",
            "4. close": "19.1600",
            "5. volume": "1500"
        },
        "2023-01-09 04:15:00": {
            "1. open": "19.1400",
            "2. high": "19.1400",
            "3. low": "19.1400",
            "4. close": "19.1400",
            "5. volume": "2988"
        },
        "2023-01-09 04:50:00": {
            "1. open": "19.1700",
            "2. high": "19.1700",
            "3. low": "19.1700",
            "4. close": "19.1700",
            "5. volume": "1532"
        },
        "2023-01-09 05:15:00": {
 